In [4]:
import faiss
import os
import pandas as pd
import numpy 
import glob
import numpy as np
from skimage import measure, transform, feature
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import ml_metrics as metrics
%matplotlib inline

In [34]:
# DB details
NPZ_FEAT_SRC = "/local/cs572/rchan31/codespace/BaselinePrototype/npz_file_saves/COMPACT_NPZ_DELF_FEATS/"
IMG_SRC_PATH = "/local/cs572/cziems/index/"

DB_LOOKUP_CSV = "/local/cs572/rchan31/fINALwEEK/csv_lookups/FAISS_delf_v1_CSV_lookup.csv"
N_INDEX = 952133
TOP_K = 100

In [2]:
# Query details
Q_NPZ_FEAT_SRC = "/local/cs572/rchan31/fINALwEEK/COMPACT_NPZ_DELF_test_FEATS"
Q_IMG_SRC_PATH = "/local/cs572/cziems/test/"

Q_LOOKUP_CSV = "/local/cs572/rchan31/fINALwEEK/csv_lookups/processed_retrieval_solutions.csv"
Q_N_INDEX = 667

INITIAL_RANKED_LIST_CSV = "/local/cs572/rchan31/fINALwEEK/csv_lookups/initial_retrievals_delf_v1.csv"

RERANKED_LIST_CSV = "/local/cs572/rchan31/fINALwEEK/csv_lookups/reranked_results_FOLDER/reranked_retrievals_delf_v1.csv"

In [5]:
# Load Query Reference Table
query_df = pd.read_csv(Q_LOOKUP_CSV)
query_image_IDs = query_df["id"]
print(len(query_image_IDs))
query_df.head()

667


,Unnamed: 0,id,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0,f60a52c609ab0208,03548a203d4b07f5,03a75dd20de880e3,04269e168c004405,04ceea28094f5758,054e6f31bdf12407,0555aac7133acbf1,057dc023b9414fe1,05a00237c243356c,...,3586fd1e2703fe16,35ec6ef9946e40d6,3766944fab5c3d1a,37ebb0e06f5be51f,38eba56c2498fdcf,39192a6d0d690e45,3a125000b1ea0de5,3ab34fe9d1dfcd15,3bb176e4dfd77846,3c428ddc62d721a4
1,1,4fa678d60ab4bc23,007eda76138e36e1,03edf3db543d2973,1c37cf230d45cf03,1e987a74ef9941bd,2eb8a14efa2f8724,2f3e7fe2aba15989,3869174c22bba91f,3873f493c6805bee,...,None,None,None,None,None,None,None,None,None,None
2,2,5be9d8002f8dc60e,00c4c92c7a8fd255,03a7c829d0b89c83,04c5c412e3ceec4a,0a28ea1e9de51440,0b6b90750c7b328d,0c37aad42a99d536,0df10bece1f6c632,130485c0f709ef35,...,None,None,None,None,None,None,None,None,None,None
3,3,0810084dfbce8ea8,08988c4cdb26f424,094a5e4ff1010550,421b161412e67a56,535c841fe0a36d1d,5ce9b333457b76e5,65313bf0521089c7,ace287038a3f52dd,b64fd516d9eebd5e,...,None,None,None,None,None,None,None,None,None,None
4,4,bb8ca55b60e8cff4,1e1533bb6e455633,1e58c6380e4db235,373e487e2562f02e,522d403681efdfee,5d30abd250225062,7b94bb1f87773b98,7ee82dfb4eaa3668,ad2ecd8c4a2d02d1,...,None,None,None,None,None,None,None,None,None,None


In [6]:
# Load Database Reference Table
db_df = pd.read_csv(DB_LOOKUP_CSV)
db_df.head()

,Unnamed: 0,filenames
0,0,00002469b818f290
1,1,0000298d976221f3
2,2,00002e4e382333bf
3,3,000036329c35b65c
4,4,0000394edaaa55b1


In [27]:
# Load Initial Ranked List Table

init_rank_df = pd.read_csv(INITIAL_RANKED_LIST_CSV)
# init_rank_df.reset_index("queryImageID")
init_rank_df.head()

,Unnamed: 0,queryImageID,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0,f60a52c609ab0208,c62a9ca23e45420f,61f08305fbc9f806,ff039f52ca878504,f258c7eb40fdc8cf,7c9c6f52fd38e29b,5cbb626ced45ba96,9ae068672e7d8270,61e5ac7b27f26e15,...,8e0eaa513a8af4db,d27ece3ea4f076eb,ceac48e131c8b88d,2389c6f3ae0d098c,f82730e8dac3257e,dc7854a19ce80576,b9c367d64335ecd3,c1a50fc8a5713159,0bb3b80c15238c02,e4d4026892f0f21a
1,1,4fa678d60ab4bc23,86b37c8c6c559b21,921f02216a181a18,9adc544d34892d6a,720382171af17c2f,8b21751541e80559,e25485a00530097d,5f443f06cae15cd5,3f84e9a136feffa2,...,caddb647d2cbaf39,47367775e52cd791,735ed8d385f4d91e,25ec89b5e07e16b8,9d8dfe0b2e7d772d,348c5d5e6ba34483,58462343843646b0,a7b375b6a66d338c,201e24b5cea6c46d,777ef9d9e67e75b8
2,2,5be9d8002f8dc60e,3dcfb68630dae905,d0abe024e8e92459,6e709c5b726a7527,4327234a4d27d181,eebf7fc00415e13e,fc596e3a813e548b,899fc319f90c23e3,190b40938e2bd088,...,01adea145c9c88c7,4d6014220144c17f,96d55734414382af,317fd548bf0ce7ea,fab2be691667f47c,9ef0c17f26803ead,6b05227e25fdb494,d18e8002a1b905aa,a012ef3d8a344176,ad60afea0399f43b
3,3,0810084dfbce8ea8,70e8730fe483d70c,5114c8e3265ada3d,aeba6ccde4c770ff,3330b87594b7dbfc,972862b79aa07753,0944c9f723b5b4ed,8307c5fff7779d70,5d48cd80a39de57c,...,e145410873532e33,b6c21875e44aafad,645c68dc24280ff7,a392d62ee37ed23f,1f901aa928f363f9,e763186d9699dfdf,24e70e3c2250dfdf,2caaf7b48d2d2075,ad12758119ab6331,25639c8e403a3827
4,4,bb8ca55b60e8cff4,e714bc41d5d7084c,d6433b3ad4f2258f,db518a09119c7b1f,0d37fbeeac8a6534,2214d68952b1c5d1,1e897b3ee5a22f43,738bd050be438d37,71b470bc516392da,...,0cd02ad748bc87ca,3e6230a8ffd06118,7677994e9d7eb07f,dac8cd061216191d,c3cf76113dee4e73,06c0a98dd640b970,7666863f158d4434,75ab910f37a7ed3d,3fff27ee908d12d1,be06e9d1a41cd538


In [28]:
init_rank_df.drop(init_rank_df.columns[0], axis=1, inplace=True)
init_rank_df.set_index("queryImageID", drop=True, inplace = True)


init_rank_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
queryImageID,,,,,,,,,,,,,,,,,,,,,
f60a52c609ab0208,c62a9ca23e45420f,61f08305fbc9f806,ff039f52ca878504,f258c7eb40fdc8cf,7c9c6f52fd38e29b,5cbb626ced45ba96,9ae068672e7d8270,61e5ac7b27f26e15,c800b2f0e4d23943,d54d45d1931a54c5,...,8e0eaa513a8af4db,d27ece3ea4f076eb,ceac48e131c8b88d,2389c6f3ae0d098c,f82730e8dac3257e,dc7854a19ce80576,b9c367d64335ecd3,c1a50fc8a5713159,0bb3b80c15238c02,e4d4026892f0f21a
4fa678d60ab4bc23,86b37c8c6c559b21,921f02216a181a18,9adc544d34892d6a,720382171af17c2f,8b21751541e80559,e25485a00530097d,5f443f06cae15cd5,3f84e9a136feffa2,d2726ef324ceb7e8,1e499b80114c29d2,...,caddb647d2cbaf39,47367775e52cd791,735ed8d385f4d91e,25ec89b5e07e16b8,9d8dfe0b2e7d772d,348c5d5e6ba34483,58462343843646b0,a7b375b6a66d338c,201e24b5cea6c46d,777ef9d9e67e75b8
5be9d8002f8dc60e,3dcfb68630dae905,d0abe024e8e92459,6e709c5b726a7527,4327234a4d27d181,eebf7fc00415e13e,fc596e3a813e548b,899fc319f90c23e3,190b40938e2bd088,a6ee7fbd054d8e16,f25719080417dc30,...,01adea145c9c88c7,4d6014220144c17f,96d55734414382af,317fd548bf0ce7ea,fab2be691667f47c,9ef0c17f26803ead,6b05227e25fdb494,d18e8002a1b905aa,a012ef3d8a344176,ad60afea0399f43b
0810084dfbce8ea8,70e8730fe483d70c,5114c8e3265ada3d,aeba6ccde4c770ff,3330b87594b7dbfc,972862b79aa07753,0944c9f723b5b4ed,8307c5fff7779d70,5d48cd80a39de57c,fa94b601d2382721,8b12dee8701b432c,...,e145410873532e33,b6c21875e44aafad,645c68dc24280ff7,a392d62ee37ed23f,1f901aa928f363f9,e763186d9699dfdf,24e70e3c2250dfdf,2caaf7b48d2d2075,ad12758119ab6331,25639c8e403a3827
bb8ca55b60e8cff4,e714bc41d5d7084c,d6433b3ad4f2258f,db518a09119c7b1f,0d37fbeeac8a6534,2214d68952b1c5d1,1e897b3ee5a22f43,738bd050be438d37,71b470bc516392da,4de111f0414cadd0,b62129fee983e2a4,...,0cd02ad748bc87ca,3e6230a8ffd06118,7677994e9d7eb07f,dac8cd061216191d,c3cf76113dee4e73,06c0a98dd640b970,7666863f158d4434,75ab910f37a7ed3d,3fff27ee908d12d1,be06e9d1a41cd538


In [58]:
def rerank_retrievals(q_idx, query_id, ranked_list): # per query
    
    # =================== reranking part ===================
    query_npz_path = os.path.join(Q_NPZ_FEAT_SRC, "%s.npz" % query_id)
    
    q_locations_x = np.asarray(np.load(query_npz_path)["locations_x"], np.int32)
    q_locations_y = np.asarray(np.load(query_npz_path)["locations_y"], np.int32)
    q_locations = np.asarray([q_locations_x, q_locations_y]).T
    
    num_features_q = q_locations.shape[0]
    # ======================================================
    
    query_db_inliers = {}
    db_ret_img_IDs = []
    return_list = []
    for topk_idx, ranked_fileID in enumerate(ranked_list):
        
        if ranked_fileID == None: # taking care of -1 retrieved images
            continue
        
        # =================== reranking part ===================
        db_npz_filename_path = os.path.join(NPZ_FEAT_SRC, "%s.npz" % ranked_fileID)
        insta_imageID = "%s.jpg" % ranked_fileID
        npz_data = np.load(db_npz_filename_path)
        
        inst_location_x = np.asarray(npz_data["locations_x"], np.int32)
        inst_location_y = np.asarray(npz_data["locations_y"], np.int32)
        db_locations = np.asarray([inst_location_x, inst_location_y]).T

        num_features_db = db_locations.shape[0]
        
        min_feat_len = min(num_features_q, num_features_db)


        # Perform geometric verification using RANSAC.
        _, inliers = measure.ransac((db_locations[:min_feat_len, :],
                                     q_locations[:min_feat_len, :]),
                                      transform.AffineTransform,
                                      min_samples=3,
                                      residual_threshold=20,
                                      max_trials=1000)

        num_inliers = 0
        if inliers is not None:
            num_inliers = sum(inliers)
        print('Found %d inliers for image : %s.jpg' % (num_inliers, ranked_fileID))
        query_db_inliers[ranked_fileID] = num_inliers
        db_ret_img_IDs.append(ranked_fileID)
        # ======================================================
        
        if topk_idx == 2:
            break
    
    # =================== reranking part ===================
    db_ret_img_IDs.sort(key=query_db_inliers.get, reverse = True)  
    # ======================================================
    
#     if len(db_ret_img_IDs) != TOP_K:
#         db_ret_img_IDs = db_ret_img_IDs + ['None'] * (TOP_K-len(db_ret_img_IDs))
        
    return_list = ["%d-%s" % (val, i) for i, val in zip(db_ret_img_IDs, sorted(query_db_inliers.values(), reverse=True))]
    if len(return_list) != TOP_K:
        return_list = return_list + ['None'] * (TOP_K-len(return_list))
        
    return return_list


re_ranked_q_db_pairs = []
for q_idx, query_row in query_df.iterrows():
    
    db_ret_img_IDs = rerank_retrievals(q_idx, query_row["id"], init_rank_df.loc[query_row["id"]])
    
    re_ranked_q_db_pairs.append(db_ret_img_IDs)
    
    print("............ Finished retrieving for query : %s.jpg : %d / %d" % (query_row["id"], (q_idx + 1), query_df.shape[0]))
    
    
    if q_idx % 1 == 0:
        reranked_retrieval_df = pd.DataFrame(re_ranked_q_db_pairs)
        reranked_retrieval_df.insert(0, "queryImageID", query_df["id"].loc[:q_idx + 1])
        reranked_retrieval_df.to_csv(RERANKED_LIST_CSV)
    
    if q_idx == 2:
        break

Found 11 inliers for image : c62a9ca23e45420f.jpg
Found 10 inliers for image : 61f08305fbc9f806.jpg
Found 9 inliers for image : ff039f52ca878504.jpg
............ Finished retrieving for query : f60a52c609ab0208.jpg : 1 / 667
Found 14 inliers for image : 86b37c8c6c559b21.jpg
Found 16 inliers for image : 921f02216a181a18.jpg
Found 16 inliers for image : 9adc544d34892d6a.jpg
............ Finished retrieving for query : 4fa678d60ab4bc23.jpg : 2 / 667
Found 18 inliers for image : 3dcfb68630dae905.jpg
Found 14 inliers for image : d0abe024e8e92459.jpg
Found 14 inliers for image : 6e709c5b726a7527.jpg
............ Finished retrieving for query : 5be9d8002f8dc60e.jpg : 3 / 667


In [59]:

reranked_retrieval_df.head()

,queryImageID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,f60a52c609ab0208,11-c62a9ca23e45420f,10-61f08305fbc9f806,9-ff039f52ca878504,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,4fa678d60ab4bc23,16-921f02216a181a18,16-9adc544d34892d6a,14-86b37c8c6c559b21,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,5be9d8002f8dc60e,18-3dcfb68630dae905,14-d0abe024e8e92459,14-6e709c5b726a7527,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [53]:
# ## Removing first-columns : imageID
# # cropping out only the retrieved-db-image IDs
# retrieved_solution = retrieval_df.iloc[:, 1:] 
# retrieved_solution = retrieved_solution.values.tolist()
# print(len(retrieved_solution[0]))
# len(retrieved_solution)

In [54]:
# ## Removing first-columns : imageID
# # cropping out only the q-image IDs
# solution_df = pd.read_csv(Q_LOOKUP_CSV)
# grountruth_solution = solution_df.iloc[:, 2:]
# grountruth_solution = grountruth_solution.values.tolist()
# print(len(grountruth_solution[0]))
# len(grountruth_solution)

In [55]:
# mAPk_cum = 0
# TOP_K = 100
# for q in range(len(retrieved_solution)):
#     temp_mAP = metrics.mapk(grountruth_solution[q], retrieved_solution[q], TOP_K)
#     print("mAP for query index :%s : %s" % (q, temp_mAP))
#     mAPk_cum += temp_mAP

# mAPk_avg = mAPk_cum/len(retrieved_solution)
# print(mAPk_cum)
# print("mAP across first all queries : ", mAPk_avg) # 0.1916305682880129